<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/PerformSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

De Serialze the saved model

In [61]:

#Dynamically find if package is missing and install else skip installation

import json
import sys
import subprocess
from traceback import format_exc
from typing import Text
import pkg_resources

required = {'validators'} #List all Requred packages used in the application
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)


!pip3 install gensim
!pip3 install translate

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
outputFolder='/content/gdrive/My Drive/DissertationFiles/'

Mounted at /content/gdrive


In [33]:
import pandas as pd
csvWritePath = outputFolder+'pandasDataFrame.csv'
# #Read Dataframe from CSV file
urlDetails = pd.read_csv(csvWritePath, sep=',', encoding='utf8')

In [5]:
urlDetails.head()

,Url,TextInfo,tokenizedWords,tokenizedScentence,POS
0,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung 1980,"['kabinettsprotokolle', 'bundesregierung', '19...",kabinettsprotokolle bundesregierung 1980,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
1,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 7 195...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
2,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 21 19...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
3,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 25 19...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
4,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 19 19...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."


In [62]:
import gensim

tfidf_corpus = gensim.corpora.MmCorpus(outputFolder+'tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus(outputFolder+'lsi_model_mm')
lsi_model = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms)
print(tfidf_corpus)
print(lsi_corpus)

MmCorpus(61 documents, 3081 features, 5182 non-zero entries)
MmCorpus(61 documents, 55 features, 2494 non-zero entries)


In [7]:
from gensim.similarities import MatrixSimilarity

search_index = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms)

In [8]:
urlDetails.head()

,Url,TextInfo,tokenizedWords,tokenizedScentence,POS
0,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung 1980,"['kabinettsprotokolle', 'bundesregierung', '19...",kabinettsprotokolle bundesregierung 1980,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
1,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 7 195...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
2,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 21 19...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
3,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 25 19...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."
4,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...,"['kabinettsprotokolle', 'bundesregierung', 'ba...",kabinettsprotokolle bundesregierung band 19 19...,"['kabinettsprotokolle_NN', 'bundesregierung_NN..."


In [42]:
def ConvertStringToArr(strVal):
  return strVal.strip("][").replace("'",'').split(', ')

In [43]:
urlDetails["tokenizedWords"] = urlDetails["tokenizedWords"].apply(ConvertStringToArr)


In [46]:
urlDetails["POS"] = urlDetails["POS"].apply(ConvertStringToArr)

'kabinettsprotokolle bundesregierung 1987'

In [45]:
print(type(urlDetails["tokenizedWords"][0]))

<class 'list'>


In [51]:

def GetAllItems(arr):
  allItems = []
  # arr =rows.to_list()
  for item in arr:
    # print(type(item))
    # allItems.append(item)
    allItems.append(item)
  return allItems  

allWords = GetAllItems(urlDetails["tokenizedWords"].to_list())
allWords[0]
print(type(allWords[0]))
# print(allWords[1])
# print(len(allWords))


<class 'list'>


In [54]:
from gensim import corpora

allWords = GetAllItems(urlDetails.tokenizedWords)
print(allWords[10])
print(len(allWords))

#creating term dictionary
dictionary = corpora.Dictionary(allWords)
#filterout if not present in more than 4 pages
# dictionary.filter_extremes(no_below=4, no_above=0.2)
#Remove unwanted words from dictionary
stoplist = set('de')
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)


['allgemeine', 'hinweisehier', 'finden', 'informationen', 'hilfe', 'benutzung', 'internetpräsentation', 'kabinettsprotokolle', 'sowie', 'impressum']
61


In [59]:
from operator import itemgetter
import nltk


def search_similar_text(search_term):

    query_bow = dictionary.doc2bow(nltk.word_tokenize(search_term))
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]

    search_index.num_best = 5

    search_list = search_index[query_lsi]

    search_list.sort(key=itemgetter(1), reverse=True)
    search_names = []

    for j, searchResult in enumerate(search_list):
      relavenceFactor =round((searchResult[1] * 100),2)
      if relavenceFactor > 2:
          search_names.append (
              {
                  'Relevance': round((searchResult[1] * 100),2),
                  'Url': urlDetails['Url'][searchResult[0]],
                  'Text': urlDetails['TextInfo'][searchResult[0]]
              }

          )
          if j == (search_index.num_best-1):
              break

    return pd.DataFrame(search_names, columns=['Relevance','Url','Text'])

ModuleNotFoundError: ignored

In [63]:
df = search_similar_text('Bildungsnotstand konfrontiert')
df.head()

LookupError: ignored

In [64]:
from translate import Translator


def translateTextToEnglish(text2Trans):
  translator= Translator(from_lang="de", to_lang="en")
  translation = translator.translate(text2Trans).replace("&#39;","'")
  return translation


def translateTextToGerman(text2Trans):
  translator= Translator(from_lang="en", to_lang="de")
  translation = translator.translate(text2Trans).replace("&#39;","'")
  return translation
    

In [65]:
urlDetails.tokenizedScentence[10]


'allgemeine hinweisehier finden informationen hilfe benutzung internetpräsentation kabinettsprotokolle sowie impressum'

In [67]:
translateTextToEnglish(urlDetails.tokenizedScentence[10])


'find general information here information help use internet presentation cabinet minutes and imprint'

In [68]:
translateTextToGerman(translateTextToEnglish(urlDetails.tokenizedScentence[10]))

'Allgemeine Informationen finden Sie hier Informationen Hilfe Internetpräsentation Kabinett Protokolle und Impressum'